In [24]:
import os
import sys
import numpy as np
from scipy.sparse import csr_matrix
from mnick_rescal import als
from triple_reader import triple_reader
from question_reader import question_reader
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
# MUST USE WORD2VEC, FIND EITHER ENTITY-ENTITY OR ENTITY-RELATION that are closest
# from gensim import utils
from gensim.models import word2vec

In [2]:
# initialize word2vec model
word_model = word2vec.Word2Vec.load_word2vec_format('word2vec.bin', binary=True)

In [33]:
# read in file as tensors
text_file = ("/Users/SaahilM/Documents/Princeton/Academics/Thesis/"
    "Senior Thesis Code/ModifiedEntityGraph/prod/MCTest/production/MCTest/OCR_text/1/Triples/1-long2.txt")
    
tr = triple_reader(text_file)
print tr.tripleList
tensor = tr.tensor
enMap = tr.enMap
relMap = tr.relMap

[['I' 'describes' 'behavior']
 ['narrator' 'describes' 'behavior']
 ['he' 'rooming' 'behavior']
 ['narrator' 'describes' 'couple']
 ['narrator' 'describes' 'himself']
 ['Jerry' 'deceitful' 'damage']
 ['Jerry' 'was' 'deceitful']
 ['Jerry' 'was' 'deceitful']
 ['I' 'think' 'damage']
 ['I' 'was' 'think']
 ['I' 'was' 'think']
 ['he' 'imaginative' 'damage']
 ['he' 'was' 'imaginative']
 ['he' 'was' 'imaginative']
 ['his' 'was' 'life']
 ['people' 'was' 'led']
 ['ambition' 'makes' 'ambitions']
 ['ambition' 'makes' 'liars']
 ['Jerry' 'had' 'ambitions']
 ['Jerry' 'had' 'liars']
 ['egotism' 'does' 'ambitions']
 ['egotism' 'does' 'liars']
 ['Jerry' 'was' 'careful']
 ['he' 'was' 'Boston']
 ['He' 'was' 'said']
 ['he' 'told' 'me']
 ['he' 'was' 'told']
 ['I' 'Medford' 'me']
 ['I' 'was' 'Medford']
 ['I' 'said' 'me']
 ['I' 'was' 'said']
 ['passport' 'said' 'Watertown']
 ['He' 'felt' 'it']
 ['he' 'had' 'it']
 ['That' 'explained' 'lot']
 ['it' 'are' 'seems']
 ['we' 'are' 'make']
 ['We' 'are' 'class-conscio

In [34]:
print len(tensor)
print tensor[0].shape
# convert them to scipy csr_matrix
csrTensor = map(lambda x: csr_matrix(x), tensor)

A, R, f, itr, exectimes = als(csrTensor, 20)
X_tilde = []
for k in range(len(R)):
    X_tilde.append(A.dot(R[k].dot(A.T)))

print A
print R
print f
print itr
print exectimes
print X_tilde
print A.shape
print np.array(X_tilde).shape
print np.array(tensor).shape

55
(110, 110)
[[ -2.98876911e-02   5.45373534e-02  -5.42379949e-02 ...,  -2.55287991e-01
    3.01288483e-02  -3.47177837e-02]
 [ -5.10474311e-11  -1.67648324e-11   1.73481801e-09 ...,  -2.58175570e-10
    4.41781855e-10  -1.07995466e-10]
 [  8.05286989e-13   1.29388081e-13  -1.32716101e-12 ...,   5.91575990e-13
   -1.61522940e-12   4.68233614e-13]
 ..., 
 [  1.44738648e-66  -1.20191791e-67  -1.67335686e-65 ...,   8.04068066e-67
   -4.90960845e-66   1.30541974e-66]
 [ -4.33613075e-02  -7.58185825e-02  -2.33260122e-02 ...,   1.15118843e-01
    4.38439191e-02  -4.31378934e-02]
 [ -9.00612814e-27   8.46757301e-27   1.64866523e-25 ...,   3.33128814e-26
    1.94760601e-26  -9.17230387e-27]]
[array([[ -9.86199971e-04,  -1.14793582e-03,  -7.57095806e-03,
          1.32193467e-03,   7.61470910e-04,  -9.56470023e-04,
         -1.01707352e-16,   2.42678565e-04,   2.33464716e-02,
         -1.68518184e-03,   1.84351357e-02,   9.06362975e-03,
          7.74282671e-04,   9.24442948e-17,   1.34900276e

In [35]:
# Find the word in entity that's most similar to given word
# default topsim to lowest possible python int
def findSimEn(word):
    topsim = None
    topEn = None
    for en in enMap:
        if type(en) == int:
            continue
        try:
            sim = word_model.similarity(en, word)
        except KeyError as e:
            sim = None
        if sim > topsim:
            topsim = sim
            topEn = en
    return [topEn, topsim]

def findSimRel(word):
    topsim = None
    topRel = None
    for rel in relMap:
        if type(rel) == int:
            continue
        try:
            sim = word_model.similarity(rel, word)
        except KeyError as e:
            sim = None
        if sim > topsim:
            topsim = sim
            topRel = rel
    return [topRel, topsim]

print findSimEn('critic')
print findSimRel('go')

# Return top similarity, sim score, and isEn boolean
def findTopEnOrRel(word):
#     try:
    [topEn, topEnsim] = findSimEn(word)
    [topRel, topRelsim] = findSimRel(word)
    if topEnsim > topRelsim:
        return (topEn, topEnsim, True, word)
    else:
        return (topRel, topRelsim, False, word)
        
    # if can't find similarity, ignore it
#     except KeyError as e:
#         return [None, 0, False, word]
    
# Return top 2 sims for an array of words
def findTopEnOrRelArr(wordArr):
    topArrs = []
    for word in wordArr:
        top = findTopEnOrRel(word)
#         print top
        topArrs.append(top)
#     print topArrs
#     print topArrs
    sortedTop = sorted(topArrs, key=lambda x: -x[1] if x[1] is not None else sys.maxint)
#     print sortedTop
    top1 = sortedTop[0]
    top2 = None
    # if the top is a relation, we have to pick an entity
    if top1[2] == False:
        curIndex = 1
        while curIndex < len(sortedTop):
            cur = sortedTop[curIndex]
            if cur[2] == False:
                curIndex += 1
                continue
            else:
                top2 = cur
                break
    else:
        top2 = sortedTop[1]
    if top2 == None:
        top2 = [0, 0, True]
    return [top1, top2]

print findTopEnOrRelArr(["critic", "went", "hi", "boat", "paint"])

['distaste', 0.26808481582806276]
['do', 0.54376041435035027]
[('saw', 0.52656813492727217, False, 'went'), ('distaste', 0.26808481582806276, True, 'critic')]


In [36]:
# for each question choice, find the entity/relation that is most-matched in question 
# then we find the choice that matches the result the most

q_file = ("/Users/SaahilM/Documents/Princeton/Academics/Thesis/Senior Thesis Code/"
"ModifiedEntityGraph/prod/MCTest/production/MCTest/OCR_text/1/1-long2-q.txt")
qr = question_reader(q_file)
questions = qr.questionCombos
print qr.numQuestions
print len(questions)
for question in questions:
    question_words = question[0].split(" ")
    print question_words
    # remove stopwords
    question_words = [word for word in question_words if word not in stopwords.words('english')]
#     print question_words
#     print str(question_words)
    [top1, top2] = findTopEnOrRelArr(question_words)
    print [top1, top2]
    
    # if word in question doesn't match, move on
    if top1[0] == None:
        continue
    
    wordMatch = None
    # If both entities go through X_tilde, find which relation has highest score
    if top1[2] == True and top2[2] == True:
        top1EnInt = enMap[top1[0]]
        top2EnInt = enMap[top2[0]]
        max_rel = None
        max_score = None
        for rel in xrange(len(X_tilde)):
            cur_score = X_tilde[rel][top1EnInt][top2EnInt]
            if cur_score > max_score:
                max_score = cur_score
                max_rel = rel
        #Now find which choice matches most with relWord
        relWord = relMap[max_rel]
        wordMatch = relWord
        
    # If entity and relation, find which e_2 has highest score
    elif top1[2] == True and top2[2] == False:
        top1EnInt = enMap[top1[0]]
        top2RelInt = relMap[top2[0]]
        max_en = None
        max_score = None
        for en in xrange(len(X_tilde[top2RelInt][top1EnInt])):
            cur_score = X_tilde[top2RelInt][top1EnInt][en]
            if cur_score > max_score:
                max_score = cur_score
                max_en = en
        enWord = enMap[max_en]
        wordMatch = enWord
    elif top1[2] == False and top2[2] == True:
        top2EnInt = enMap[top2[0]]
        top1RelInt = relMap[top1[0]]
        max_en = None
        max_score = None
        for en in xrange(len(X_tilde[top1RelInt][top2EnInt])):
            cur_score = X_tilde[top1RelInt][top2EnInt][en]
            if cur_score > max_score:
                max_score = cur_score
                max_en = en
        enWord = enMap[max_en]
        wordMatch = enWord
    else:
        raise Exception("shouldn't get here!")
    
    print wordMatch
    # for each choice, iterate through the words in the choice, find the choice with highest similarity
    choices = question[1:]
    max_sim = None
    max_choice = None
    for i in xrange(len(choices)):
        choice = choices[i]
        choice_words = choice.split(" ")
        question_words = [word for word in question_words if word not in stopwords.words('english')]
        for word in choice_words:
            try:
                sim = word_model.similarity(word, wordMatch)
            except KeyError as e:
                sim = 0
            if sim > max_sim:
                max_sim = sim
                max_choice = i
    if max_choice == 0:
        max_choice = "A"
    if max_choice == 1:
        max_choice = "B"
    if max_choice == 2:
        max_choice = "C"
    if max_choice == 3:
        max_choice = "D"
    if max_choice == 4:
        max_choice = "E"
    print max_choice

numQuestions  13
13
13
[u'Jerry', u'in', u'Passage', u'1', u'and', u'the', u'narrator', u'of', u'Passage', u'2', u'are', u'similar', u'in', u'that', u'both']
[('Jerry', 1.0, True, u'Jerry'), ('narrator', 1.0, True, u'narrator')]
wanted
A
[u'Jerry', u'differs', u'most', u'from', u'the', u'narrator', u'of', u'Passage', u'2', u'in', u'his']
[('Jerry', 1.0, True, u'Jerry'), ('narrator', 1.0, True, u'narrator')]
wanted
C
[u'The', u'first', u'sentence', u'of', u'Passage', u'1', u'implies', u'that']
[('This', 0.7025192721804896, True, u'The'), ('means', 0.62676256128193342, False, u'implies')]
lives
E
[u'In', u'line', u'6,', u'"modest"', u'most', u'nearly', u'means']
[('means', 1.0, False, u'means'), ('This', 0.42479429618445708, True, u'In')]
lives
D
[u'In', u'the', u'context', u'of', u'Passage', u'1,', u'"insignificant"', u'(line', u'21)', u'suggests', u'that']
[('seems', 0.50263214831747294, True, u'suggests'), ('This', 0.42479429618445708, True, u'In')]
seem
B
[u'Passage', u'1', u'indicat